In [10]:
import pandas as pd

# Define column names and read CSV file
colnames = ['Date & Time', 'Location', 'Names', 'Orders', 'Total Amount', 'Payment Option', 'Card Number']
df = pd.read_csv("chesterfield_25-08-2021_09-00-00.csv", names=colnames, header=None)

In [11]:
# Remove unnecessary and sensitive data columns
columns_to_drop_index = [2,6]
df.drop(columns=df.columns[columns_to_drop_index], axis=1, inplace=True)

# Remove duplicate records
df.drop_duplicates(inplace=True)

# Remove rows with missing value
df.dropna(inplace=True)

In [12]:
# Convert Total Amount column to float and remove rows with invalid floats
df['Total Amount'] = pd.to_numeric(df['Total Amount'], errors='coerce')
df.dropna(subset=['Total Amount'], inplace=True)

In [13]:
# Split orders and explode them
df['Orders'] = df['Orders'].str.split(',')
df = df.explode('Orders')
df.reset_index(drop=True, inplace=True)

# Remove product prices from the text and extract product prices
df['Product'] = df['Orders'].str.replace(r'\s*\d+\.\d+\s*$', '').str.strip()
df['Product Price'] = df['Orders'].str.extract(r'(\d+\.\d+)')

# Drop the original text column
df.drop(columns=['Orders'], inplace=True)

# Clean product names and convert prices to float
df['Product'] = df['Product'].str.rstrip('-')
df['Product Price'] = df['Product Price'].astype(float)

# Stripping price from exploded products
df['Product'] = df['Product'].str.rstrip('- 0123456789.')

# Dropping duplicates from table
df.drop_duplicates(inplace=True)

# Add white spaces back to product names
df['Product'] = df['Product'].str.replace(r'(?<=[a-z])(?=[A-Z])', ' ')


In [14]:
# Rename Columns
df.rename(columns={'Product Price': 'Product Price(£)'}, inplace=True)
df.rename(columns={'Total Amount': 'Total Amount(£)'}, inplace=True)

In [15]:
df

,Date & Time,Location,Total Amount(£),Payment Option,Product,Product Price(£)
0,25/08/2021 09:00,Chesterfield,5.20,CARD,Regular Flavoured iced latte - Hazelnut,2.75
1,25/08/2021 09:00,Chesterfield,5.20,CARD,Large Latte,2.45
2,25/08/2021 09:02,Chesterfield,17.30,CARD,Large Flavoured iced latte - Caramel,3.25
3,25/08/2021 09:02,Chesterfield,17.30,CARD,Regular Flavoured iced latte - Hazelnut,2.75
4,25/08/2021 09:02,Chesterfield,17.30,CARD,Regular Flavoured iced latte - Caramel,2.75
...,...,...,...,...,...,...
774,25/08/2021 16:52,Chesterfield,10.70,CASH,Regular Flavoured iced latte - Caramel,2.75
775,25/08/2021 16:52,Chesterfield,10.70,CASH,Regular Flavoured latte - Hazelnut,2.55
776,25/08/2021 16:52,Chesterfield,10.70,CASH,Regular Flat white,2.15
777,25/08/2021 16:56,Chesterfield,2.55,CASH,Regular Flavoured latte - Hazelnut,2.55
